# RNN网络生成随机名字

In [1]:
# COPYRIGHT BY EX10SI0N @ www.github.com/Ex10si0n

## 数据采集处理

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
# 处理读入文件为str类型
source = open("namesrc.txt").read()

In [4]:
# 处理source中的unique
source_set = sorted(set(source))

In [5]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(source_set)}
idx2char = np.array(source_set)
text_as_int = np.array([char2idx[c] for c in source])

In [6]:
# 索引表打印测试
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  '业' :   1,
  '东' :   2,
  '久' :   3,
  '义' :   4,
  '乐' :   5,
  '云' :   6,
  '亚' :   7,
  '亦' :   8,
  '京' :   9,
  '亭' :  10,
  '仁' :  11,
  '仑' :  12,
  '仟' :  13,
  '仲' :  14,
  '伟' :  15,
  '伦' :  16,
  '佐' :  17,
  '佑' :  18,
  '佳' :  19,
  ...
}


## 创建训练样本和目标

In [7]:
seq_length = 2
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

华
君
 
子
睿


In [8]:
seq = char_dataset.batch(seq_length + 1, drop_remainder = True)
for item in seq.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'华君 '
'子睿 '
'宏君 '
'旦宇 '
'甫宇 '


In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = seq.map(split_input_target)

In [10]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '华君'
Target data: '君 '


In [11]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 49 ('华')
  expected output: 57 ('君')
Step    1
  input: 57 ('君')
  expected output: 0 (' ')


In [12]:
# 设定批和缓冲区大小
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 2), (64, 2)), types: (tf.int64, tf.int64)>

## 创建模型

In [13]:
set_size = len(source_set)
embedding_dim = 256
rnn_units = 1024

In [14]:
def build_model(set_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(set_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(set_size)
  ])
  return model

In [15]:
model = build_model(
  set_size = len(source_set),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, set_size)")

(64, 2, 476) # (batch_size, sequence_length, set_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           121856    
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 476)           487900    
Total params: 4,548,060
Trainable params: 4,548,060
Non-trainable params: 0
_________________________________________________________________


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [19]:
sampled_indices

array([216, 464])

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 2, 476)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       6.1660275


In [21]:
model.compile(optimizer='adam', loss=loss)

## 配置检查点

In [22]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS=10

In [26]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 81 steps
Epoch 1/10
81/81 [==============================] - 4s 45ms/step - loss: 1.9940
Epoch 2/10
81/81 [==============================] - 4s 47ms/step - loss: 1.9750
Epoch 3/10
81/81 [==============================] - 4s 48ms/step - loss: 1.9606
Epoch 4/10
81/81 [==============================] - 4s 48ms/step - loss: 1.9548
Epoch 5/10
81/81 [==============================] - 4s 47ms/step - loss: 1.9467
Epoch 6/10
81/81 [==============================] - 4s 48ms/step - loss: 1.9450
Epoch 7/10
81/81 [==============================] - 4s 47ms/step - loss: 1.9367
Epoch 8/10
81/81 [==============================] - 4s 46ms/step - loss: 1.9388
Epoch 9/10
81/81 [==============================] - 4s 46ms/step - loss: 1.9269
Epoch 10/10
81/81 [==============================] - 4s 49ms/step - loss: 1.9257


In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [28]:
model = build_model(set_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            121856    
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 476)            487900    
Total params: 4,548,060
Trainable params: 4,548,060
Non-trainable params: 0
_________________________________________________________________


In [257]:
def generate_text(model, start_string):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 3.0

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

## 优化输出

In [258]:
import re
name_arr = generate_text(model, start_string=u"卓")
name_arr = name_arr.replace(" ","")
interval = re.compile('.{2}')
lastname = ' '.join(interval.findall(name_arr))
lastname = lastname.split()

## 姓氏处理（数据来自百家姓）
概率模型: `p(x) = e^(-x * (4 / 97))`

In [259]:
import random, math, csv
surname = open("surnamesrc.txt").read()
surname = surname.split()
p = [math.exp(- x * (4 / 97)) for x in range(1, len(surname))]
row = random.random()
surname_set = []
for j in range(len(name_arr)):
    for i in range(len(surname) - 1):
        if p[i] <= row:
            surname_set.append(surname[i])
            row = random.random()
            break

In [260]:
n = min(len(surname), len(lastname))
print("result: " + str(n) + " names generated")

result: 386 names generated


In [261]:
name = []
for i in range(n):
    name.append(surname[i] + lastname[i])
print(name)

['曾卓滨', '赵熙烨', '钱远白', '孙蓓礼', '李樨晗', '周昱熊', '吴尊炎', '郑菲聪', '王乐斌', '冯立军', '陈云学', '褚云祯', '卫泓亭', '蒋科弈', '沈铮秀', '韩棋诩', '杨瑾卓', '朱贤洲', '秦科封', '尤淡焱', '许泓莓', '何宝朗', '吕宗平', '施盛谦', '张希晗', '孔飘烁', '曹珝凯', '严辰祖', '华谦奎', '金纯峰', '魏翰厚', '陶俊翔', '姜琛凡', '戚钢翰', '谢飘翊', '邹晗军', '喻超歌', '柏洋亦', '水荫成', '窦睿斌', '章侦磊', '云浩麒', '苏嘉毅', '潘棱焱', '葛昱科', '奚天易', '范瑞键', '彭伟凡', '郎豫磊', '鲁伟宗', '韦景硕', '昌教斌', '马基端', '苗量坚', '凤希箫', '花泽烁', '方曦贞', '俞凯炜', '任凯孝', '袁飞琛', '柳琛育', '酆廷鑫', '鲍炜昱', '史伟常', '唐瀚晗', '费双政', '廉坤飞', '岑政超', '薛奥俊', '雷槿渤', '贺佐颜', '倪竿影', '汤亭珅', '滕云香', '殷翰辰', '罗基鸿', '毕伦枫', '郝涛鸿', '邬炳济', '安晨然', '常斌郎', '乐旬硕', '于科莫', '时悦珝', '傅俊林', '皮蕾珝', '卞格颜', '齐桦廷', '康杰雪', '伍俞乐', '余烨辉', '元淇伟', '卜岩仲', '顾易珂', '孟洋存', '平钧诩', '黄晗纹', '和璟羿', '穆贝翔', '萧风泓', '尹晨泓', '姚子化', '邵熙熠', '湛竿麒', '汪裕祖', '祁超常', '毛豫辰', '禹炜蓓', '狄麒麒', '米瑞然', '贝冬琮', '明聪骏', '臧聪俊', '计佐运', '伏绮郎', '成文祯', '戴豪轩', '谈谨涛', '宋祖瑜', '茅超成', '庞藏智', '熊龙翰', '纪凡仁', '舒斌迪', '屈先深', '项俊温', '祝君森', '董睿扬', '梁昱景', '杜迪桦', '阮风胥', '蓝莉大', '闵远郎', '席毅睿', '季飞东', '麻科贤', '强慧廷', '贾昕庄', '路基嗣', '娄珝泓', '危琳超', '江贤跃', '童毅博'